In [1]:
# 画图工具
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.width',1000)
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties 
font_set = FontProperties(fname=r"C:\windows\fonts\MicroSoft YaHei.ttc", size=12)
# %matplotlib inline 
# auto 弹出窗口 inline 默认，在当前页显示
#指定默认字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.family']='sans-serif'
#解决负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

import math
import seaborn as sns
import warnings;warnings.filterwarnings(action='once')
import matplotlib as mpl

warnings.filterwarnings('ignore')

large=22;med=16;small=12;
params={'axes.titlesize':large,
        'legend.fontsize':med,
        'figure.figsize':(16,10),
        'axes.labelsize':med,
        'axes.titlesize':med,
        'xtick.labelsize':med,
        'ytick.labelsize':med,
        'figure.titlesize':large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style('white')
print(mpl.__version__)
print(sns.__version__)


3.3.1
0.10.1


In [2]:
from ipywidgets import widgets,Button,Label
from statsmodels.tsa.seasonal import STL
from my_functions import L_plot1_twoaxis

In [3]:
from pyecharts.charts import Bar,Line
import pyecharts.options as opts
from pyecharts.globals import ThemeType
import xlsxwriter
from pandas import ExcelWriter

In [4]:
# 导入数据
df1 = pd.read_excel(r"C:\Users\57165\Desktop\data\datas\wjl1.xlsx",sheet_name='Sheet2',dtype={'srq':str,'sxl':str,'szl':str}
,converters={'nzhzb':lambda x:round(x,2),'nzzb':lambda x:round(x,2)})
df1=df1.fillna(0)

df1.head()

,srq,sxl,szl,sdl,nzxssl,nzxsje,nzml,nzzb,nzhzb
0,0101,070901火锅丸子,0709火锅食品,07冷冻冷藏,83.9075,2099.7831,486.5544,531.74,34.81
1,0102,070901火锅丸子,0709火锅食品,07冷冻冷藏,44.7919,1126.4968,263.9356,259.08,18.27
2,0103,070901火锅丸子,0709火锅食品,07冷冻冷藏,46.8191,1119.5802,258.6094,255.27,17.61
3,0104,070901火锅丸子,0709火锅食品,07冷冻冷藏,47.1179,1077.3580,254.6716,247.75,19.19
4,0105,070901火锅丸子,0709火锅食品,07冷冻冷藏,42.6091,1086.4321,256.2005,250.66,18.32


In [5]:
# 季节性筛选函数，自动识别函数，暂时先特制
'''
Step 0:所有品类的指标值计算移动平均,分开两种，一种是1-12月，另一种是5-4月 冬季
Step 1:进行2阶曲线拟合
Step 2:计算曲线2阶导数，大于0是凹函数，无季节性，小于0 属于凸曲线，再然后加入方差大于
Step 3:筛选出的品类进行人工核对
'''
t = df1['srq'].unique()
sfl=df1['sxl'].unique()
trend=[]
for s in sfl:
    k=df1[df1['sxl']==s]['nzzb']
    stl = STL(k, period=20, robust=True)
    res_robust = stl.fit()
    
    ml=pd.DataFrame(res_robust.trend)
    ml=ml.set_index(t)
    # 归一化
    ml['diff']=ml.apply( lambda x: (x-ml.min())/(ml.max()-ml.min()) ,axis=1  )
    m2=ml.diff()
    m2['trend']=list(ml['diff'])
    m2=m2.set_index(t)

    x=m2['trend']
    x=x.fillna(0)
    i=range(1,len(x)+1)
    z=np.polyfit(i,x,2)
    p=np.poly1d(z)
   
    if z[0]<0:
        trend.append(s)
    

# k=df1[df1['sxl']]['nzzb'] # 071001冰淇淋 090402蚊香 110102实芯纸  100410护/润唇膏
# stl = STL(k, period=20, robust=True)
# res_robust = stl.fit()

In [6]:
# 冬 春 季节性，需要对数据做平移处理，即降1-3月份的数据移动到12月之后
df1['srq1']=df1['srq'].apply(lambda x : str(int(x[0:2])+12)+x[2:4])
df2=df1.copy()
df2['srq']=df2['srq1']
df3=df1[df1['srq']>='0401']
df3=df3.append(df2[df2['srq']<'1601'])
t1 = df1['srq'].unique()
sfl1=df1['sxl'].unique()

for s in sfl1:
    k=df3[df3['sxl']==s]['nzzb']
    stl = STL(k, period=20, robust=True)
    res_robust = stl.fit()
    
    ml=pd.DataFrame(res_robust.trend)
    ml=ml.set_index(t1)
    # 归一化
    ml['diff']=ml.apply( lambda x: (x-ml.min())/(ml.max()-ml.min()) ,axis=1  )
    m2=ml.diff()
    m2['trend']=list(ml['diff'])
    m2=m2.set_index(t1)

    x=m2['trend']
    x=x.fillna(0)
    i=range(1,len(x)+1)
    z=np.polyfit(i,x,2)
    p=np.poly1d(z)
   
    if z[0]<0:
        trend.append(s)
    

In [7]:
t = df1['srq'].unique()
k=df1[df1['sxl']=='100203沐浴露']['nzzb'] # 071001冰淇淋 090402蚊香 110102实芯纸  100410护/润唇膏
stl = STL(k,period=20, robust=True)
res_robust = stl.fit()


# STL 分解
ml=pd.DataFrame(res_robust.trend)
ml=ml.set_index(t)
# 归一化
ml['diff']=ml.apply( lambda x: (x-ml.min())/(ml.max()-ml.min()) ,axis=1  )
m2=ml.diff()
m2=m2.set_index(t)
# fig,ax=plt.subplots(1,1,figsize=(16,6))
# plt.plot(m2.index,m2['diff'])
# ax.set_xticks(list([t[i] for i in range(len(t)) if i%20==0]))
# ax.grid(True)
# plt.xticks(rotation=-30)
# plt.show()
m2['trend']=list(ml['diff'])
L_plot1_twoaxis(m2.index,m2)



In [8]:
trend


['071001冰淇淋',
 '090401喷雾杀虫剂',
 '090402蚊香',
 '090403电蚊器',
 '090404樟脑丸',
 '090405防蚊液',
 '090407花露水',
 '090408电蚊液',
 '090409电蚊片',
 '090410艾条/檀香',
 '090411其他',
 '100202香皂',
 '100203沐浴露',
 '100403防晒霜',
 '100405珍珠粉/面膜',
 '100407洁面乳',
 '100408气垫/粉底',
 '070901火锅丸子',
 '070902火锅肉类',
 '100202香皂',
 '100203沐浴露',
 '100401润肤霜',
 '100402润肤露',
 '100407洁面乳',
 '100408气垫/粉底',
 '100409卸妆用品',
 '100410护/润唇膏',
 '100411护手霜',
 '100412身体乳']

In [9]:
df3

,srq,sxl,szl,sdl,nzxssl,nzxsje,nzml,nzzb,nzhzb,srq1
91,0401,070901火锅丸子,0709火锅食品,07冷冻冷藏,3.5889,60.6782,12.8243,14.50,1.66,1601
92,0402,070901火锅丸子,0709火锅食品,07冷冻冷藏,10.1999,194.7651,40.0894,46.62,5.06,1602
93,0403,070901火锅丸子,0709火锅食品,07冷冻冷藏,8.6157,211.4126,37.9671,45.55,4.78,1603
94,0404,070901火锅丸子,0709火锅食品,07冷冻冷藏,10.3206,194.9439,37.7005,43.06,4.60,1604
95,0405,070901火锅丸子,0709火锅食品,07冷冻冷藏,8.5286,168.9358,32.0439,38.21,4.30,1605
...,...,...,...,...,...,...,...,...,...,...
11432,1527,110105干/湿纸巾,1101卷纸,11纸品,32.6666,129.4061,68.8166,71.57,8.48,1527
11433,1528,110105干/湿纸巾,1101卷纸,11纸品,25.6666,115.2130,49.0933,54.67,6.54,1528
11434,1529,110105干/湿纸巾,1101卷纸,11纸品,48.4000,167.6326,87.2666,102.33,12.12,1529
11435,1530,110105干/湿纸巾,1101卷纸,11纸品,43.0000,156.7662,75.1788,85.32,10.08,1530


In [10]:
k=df1[df1['sxl']=='071001冰淇淋']['nzzb']
k

1098      13.43
1099      16.07
1100       4.27
1101      10.66
1102       3.24
1103       0.00
1104       5.23
1105       2.35
1106       3.36
1107       3.49
1108       0.00
1109       9.55
1110       4.37
1111       9.14
1112       4.41
1113       4.88
1114      14.18
1115       0.00
1116       3.87
1117       2.90
1118      13.47
1119       4.99
1120      14.31
1121      20.57
1122       0.00
1123      10.18
1124       0.00
1125       0.00
1126       0.00
1127       4.45
1128       0.00
1129       0.00
1130       1.92
1131       0.00
1132       0.00
1133       6.46
1134      10.22
1135      11.76
1136      11.86
1137      15.78
1138      37.77
1139      28.30
1140      18.68
1141      12.72
1142      55.91
1143      43.78
1144       0.00
1145      52.91
1146      85.62
1147      69.37
1148      25.67
1149      49.86
1150      98.28
1151      74.01
1152      29.23
1153      12.59
1154      12.40
1155      23.19
1156      39.26
1157       0.00
1158       2.14
1159      19.36
1160    

In [11]:
t1 = df3['srq'].unique()
k1=df3[df3['sxl']=='071001冰淇淋']['nzzb'] # 071001冰淇淋 090402蚊香 110102实芯纸  100410护/润唇膏
stl1 = STL(k1, period=20, robust=True)
res_robust1 = stl1.fit()

# STL 分解
ml_1=pd.DataFrame(res_robust1.trend)
ml_1=ml_1.set_index(t1)
# 归一化
ml_1['diff']=ml_1.apply( lambda x: (x-ml_1.min())/(ml_1.max()-ml_1.min()) ,axis=1  )
m2_1=ml_1.diff()
m2_1=m2_1.set_index(t1)

m2_1['trend']=list(ml_1['diff'])
L_plot1_twoaxis(m2_1.index,m2_1)

